In [3]:
import sys
sys.path.append('../src')

import os
os.environ["LANGUAGE"] = 'ar'

from sqlalchemy.sql.expression import func
from wikidataDB import WikidataEntity, WikidataID, Session
from wikidataRetriever import WikidataKeywordSearch, AstraDBConnect
from SPARQLWrapper import SPARQLWrapper, JSON

import json
import os
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import pickle
from datasets import load_dataset
import re
from requests.exceptions import HTTPError
import time
from tqdm import tqdm

def is_in_wikipedia(qid):
    item = WikidataID.get_id(qid)
    if item is None:
        return False
    return item.in_wikipedia

In [7]:
sample_ids = pickle.load(open("../data/Evaluation Data/Sample IDs (EN).pkl", "rb"))
sample_ids = sample_ids[sample_ids['In Wikipedia']]
sample_ids = set(sample_ids['QID'].values.tolist())

In [5]:
with Session() as session:
    count = session.query(WikidataID.id).count()
count

4083442

In [6]:
with Session() as session:
    count = session.query(WikidataEntity.id).count()
count

4083245

In [ ]:
datastax_token = json.load(open(f"../API_tokens/datastax_wikidata_nvidia.json"))
graph_store = AstraDBConnect(datastax_token, 'qids_nvidia', model='nvidia', batch_size=10)

In [ ]:
BATCH_SIZE = 1000
with tqdm(total=9203786) as progressbar:
    with Session() as session:
        for i in range(2000000, 9203786, BATCH_SIZE):
            entity = session.query(WikidataEntity).join(WikidataID, WikidataEntity.id == WikidataID.id).filter(WikidataID.in_wikipedia == True).offset(i).first()
            progressbar.n = i
            progressbar.refresh()
            doc_batch = []
            ids_batch = []
            if graph_store.graph_store.get_by_document_id(f'{entity.id}_1') is None:
                    break

In [ ]:
# Function to process a single file
def process_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    processed_rows = []
    for _, data_chunks in data.items():
        for data_chunk in data_chunks:
            triple = data_chunk['triple']
            question_in_wikipedia = is_in_wikipedia(triple[0])
            answer_in_wikipedia = is_in_wikipedia(triple[2])
            question = data_chunk['question variants'][0]
            processed_rows.append({
                'Question QID': triple[0],
                'Property PID': triple[1],
                'Answer QID': triple[2],
                'Question in Wikipedia': question_in_wikipedia,
                'Answer in Wikipedia': answer_in_wikipedia,
                'Question': question['out-of-context'],
                'Answer': data_chunk['answer']
            })
    return processed_rows

# Use a list to accumulate the rows
all_data = []

main_dir = "../data/Evaluation Data/KGConv/complete_version"

# Using ThreadPoolExecutor to parallelize file processing
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = []
    for folder in os.listdir(main_dir):
        current_dir = os.path.join(main_dir, folder)
        for file in tqdm(os.listdir(current_dir)):
            file_path = os.path.join(current_dir, file)
            futures.append(executor.submit(process_file, file_path))

    for future in tqdm(futures):
        all_data.extend(future.result())

# Convert to DataFrame all at once at the end
clean_data = pd.DataFrame(all_data)
pickle.dump(clean_data, open("../data/Evaluation Data/KGConv/processed_dataframe.pkl", "wb"))

In [ ]:
# Function to process a single file
def process_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    processed_rows = []
    for data_chunk in data:
        try:
            question_qids = [d['name'] for d in data_chunk['questionEntity'] if d['entityType'] == 'entity']
            question_in_wikipedia = [is_in_wikipedia(id) for id in question_qids]

            answer_type = data_chunk['answer']['answerType']

            answer_qids = []
            answer_in_wikipedia = []
            if (answer_type == 'entity') and (data_chunk['answer']['answer'] is not None):
                answer_qids = [d['name'] for d in data_chunk['answer']['answer']]
                answer_in_wikipedia = [is_in_wikipedia(id) for id in answer_qids]

            processed_rows.append({
                'Question QIDs': question_qids,
                'Answer QIDs': answer_qids,
                'Question in Wikipedia': question_in_wikipedia,
                'Answer in Wikipedia': answer_in_wikipedia,
                'Question': data_chunk['question'],
                'Answer': data_chunk['answer']['mention'],
                'Answer Type': answer_type,
                'Language': 'en'
            })

            for lang in data_chunk['translations'].keys():
                processed_rows.append({
                    'Question QIDs': question_qids,
                    'Answer QIDs': answer_qids,
                    'Question in Wikipedia': question_in_wikipedia,
                    'Answer in Wikipedia': answer_in_wikipedia,
                    'Question': data_chunk['translations'][lang],
                    'Answer': data_chunk['answer']['mention'],
                    'Answer Type': answer_type,
                    'Language': lang
                })

        except Exception as e:
            print(data_chunk)
            raise e
    return processed_rows

# Use a list to accumulate the rows
all_data = []

main_dir = "../data/Evaluation Data/Mintaka"

# Using ThreadPoolExecutor to parallelize file processing
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = []
    for file in tqdm(os.listdir(main_dir)):
        if 'json' in file:
            file_path = os.path.join(main_dir, file)
            futures.append(executor.submit(process_file, file_path))

    for future in tqdm(futures):
        all_data.extend(future.result())

# Convert to DataFrame all at once at the end
clean_data = pd.DataFrame(all_data)
pickle.dump(clean_data, open("../data/Evaluation Data/Mintaka/processed_dataframe_full.pkl", "wb"))

In [ ]:
# Function to process a single file
def process_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    processed_rows = []
    for data_chunk in data:
        try:
            if data_chunk['question_uris'] is not None:
                question_qids = [d.split('/')[-1] for d in data_chunk['question_uris']]
                question_in_wikipedia = [is_in_wikipedia(id) for id in question_qids]

                answers = [d['wd_names']['en'][0] if len(d['wd_names']['en']) > 0 else d['label'] for d in data_chunk['answers']]
                answer_qids = [d['value'].split('/')[-1] for d in data_chunk['answers']]
                answer_in_wikipedia = [is_in_wikipedia(id) for id in answer_qids]

                processed_rows.append({
                    'Question QIDs': question_qids,
                    'Answer QIDs': answer_qids,
                    'Question in Wikipedia': question_in_wikipedia,
                    'Answer in Wikipedia': answer_in_wikipedia,
                    'Question': data_chunk['question_eng'],
                    'Answer': answers,
                })
        except Exception as e:
            print(data_chunk)
            raise e
    return processed_rows

# Use a list to accumulate the rows
all_data = []

main_dir = "../data/Evaluation Data/RuBQ"

# Using ThreadPoolExecutor to parallelize file processing
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = []
    for file in tqdm(os.listdir(main_dir)):
        file_path = os.path.join(main_dir, file)
        futures.append(executor.submit(process_file, file_path))

    for future in tqdm(futures):
        all_data.extend(future.result())

# Convert to DataFrame all at once at the end
clean_data = pd.DataFrame(all_data)
pickle.dump(clean_data, open("../data/Evaluation Data/RuBQ/processed_dataframe.pkl", "wb"))

In [ ]:
wikidata_endpoint = "https://query.wikidata.org/sparql"
sparql = SPARQLWrapper(wikidata_endpoint)
sparql.setReturnFormat(JSON)

sparql.setQuery("SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj wdt:P31 wd:Q7187 . ?sbj rdfs:label ?sbj_label . FILTER(CONTAINS(lcase(?sbj_label), 'vgf')) . FILTER (lang(?sbj_label) = 'en') } LIMIT 25")
results = sparql.query().convert()

In [ ]:
from requests.exceptions import HTTPError
import time

# Function to process a single file
def process_file(file_path):
    wikidata_endpoint = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(wikidata_endpoint)
    sparql.setReturnFormat(JSON)

    with open(file_path, 'r') as f:
        data = json.load(f)

    processed_rows = []
    for data_chunk in data:
        try:
            matches = re.findall(r'wd:Q\d+', data_chunk['sparql_wikidata'])
            question_qids = [match[3:] for match in matches]
            question_in_wikipedia = [is_in_wikipedia(id) for id in question_qids]

            answer_qids = []
            query = data_chunk['sparql_wikidata']
            ran_sparql = True
            if not data_chunk['sparql_wikidata'].lower().strip().startswith('ask'):
                results = []
                retry = 5
                ran_sparql = False
                while retry > 0:  # Retry up to 5 times
                    try:
                        sparql.setQuery(query)
                        results = sparql.query().convert()["results"]["bindings"]
                        retry = 0  # Exit loop if successful
                        ran_sparql = True
                    except HTTPError as e:
                        print(e)
                        retry -= 1
                        time.sleep(1)
                    except Exception as e:
                        print(e)
                        retry -= 1
                        query = re.sub(r'LIMIT \d+', 'LIMIT 5', query, flags=re.IGNORECASE)

                for result in results:
                    for key in result:
                        value = result[key]["value"]
                        if 'www.wikidata.org' in result[key]["value"]:
                            answer_qids.append(value.split('/')[-1])
                answer_in_wikipedia = [is_in_wikipedia(id) for id in answer_qids]

            processed_rows.append({
                'Question QIDs': question_qids,
                'Answer QIDs': answer_qids,
                'Question in Wikipedia': question_in_wikipedia,
                'Answer in Wikipedia': answer_in_wikipedia,
                'Question': data_chunk['question'],
                'SPARQL': data_chunk['sparql_wikidata'],
                'Ran SPARQL': ran_sparql
            })
        except Exception as e:
            print(e)
            print(data_chunk)
            raise e
    return processed_rows

# Use a list to accumulate the rows
all_data = []

main_dir = "../data/Evaluation Data/LC_QuAD"

# Using ThreadPoolExecutor to parallelize file processing
with ThreadPoolExecutor(max_workers=1) as executor:
    futures = []
    for file in tqdm(os.listdir(main_dir)):
        file_path = os.path.join(main_dir, file)
        futures.append(executor.submit(process_file, file_path))

    for future in tqdm(futures):
        all_data.extend(future.result())

# Convert to DataFrame all at once at the end
clean_data = pd.DataFrame(all_data)
pickle.dump(clean_data, open("../data/Evaluation Data/LC_QuAD/processed_dataframe.pkl", "wb"))

In [ ]:
clean_data = pickle.load(open("../data/Evaluation Data/LC_QuAD/processed_dataframe.pkl", "rb"))

for i, row in tqdm(clean_data.iterrows()):
    answer_qids = []
    query = row['SPARQL']
    if (not query.lower().strip().startswith('ask')) and (not row['Ran SPARQL']):
        results = []
        retry = 5
        while retry > 0:  # Retry up to 5 times
            try:
                sparql.setQuery(query)
                results = sparql.query().convert()["results"]["bindings"]
                retry = 0  # Exit loop if successful
                clean_data.at[row.name, 'Ran SPARQL'] = True
            except HTTPError as e:
                print(e)
                retry -= 1
                time.sleep(1)
            except Exception as e:
                print(e)
                retry -= 1
                query = re.sub(r'LIMIT \d+', 'LIMIT 5', query, flags=re.IGNORECASE)

        for result in results:
            for key in result:
                value = result[key]["value"]
                if 'www.wikidata.org' in result[key]["value"]:
                    answer_qids.append(value.split('/')[-1])
        answer_in_wikipedia = [is_in_wikipedia(id) for id in answer_qids]

        clean_data.at[row.name, 'Answer QIDs'] = answer_qids
        clean_data.at[row.name, 'Answer in Wikipedia'] = answer_in_wikipedia

        if i%100 == 0:
            pickle.dump(clean_data, open("../data/Evaluation Data/LC_QuAD/processed_dataframe.pkl", "wb"))

In [ ]:
# Function to process a single file
def process_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    processed_rows = []
    for data_chunk in data:
        correct_in_wikipedia = is_in_wikipedia(data_chunk['correct_id'])
        wrong_in_wikipedia = is_in_wikipedia(data_chunk['wrong_id'])
        processed_rows.append({
            'Sentence': data_chunk['text'],
            'Entity Name': data_chunk['string'],
            'Correct QID': data_chunk['correct_id'],
            'Wrong QID': data_chunk['wrong_id'],
            'Correct in Wikipedia': correct_in_wikipedia,
            'Wrong in Wikipedia': wrong_in_wikipedia,
        })
    return processed_rows

# Define main directory and path to save results
main_dir = "../data/Evaluation Data/Wikidata-Disamb"

# Use a list to accumulate the rows
all_data = []

# Iterate over each file, process it, and save in chunks
for file in tqdm(os.listdir(main_dir)):
    file_path = os.path.join(main_dir, file)
    processed_data = process_file(file_path)
    all_data.extend(processed_data)

# Convert to DataFrame all at once at the end
clean_data = pd.DataFrame(all_data)
pickle.dump(clean_data, open("../data/Evaluation Data/Wikidata-Disamb/processed_dataframe.pkl", "wb"))

In [ ]:
# Function to process a single file
def process_file(data, lang='en'):
    processed_rows = []
    for data_chunk in tqdm(data):
        qid = data_chunk['uri']
        qid_in_wikipedia = is_in_wikipedia(qid)
        boundaries = [(e['start'], e['end']) for e in data_chunk['entities'] if e['uri'] == data_chunk['uri']]
        processed_rows.append({
            'Sentence': data_chunk['text'],
            'Entity Name': data_chunk['title'],
            'Entity Span': boundaries,
            'Correct QID': qid,
            'Correct in Wikipedia': qid_in_wikipedia,
            'Language': data_chunk['lan']
        })
    return processed_rows

# Use a list to accumulate the rows
all_data = []

huggingface_ds = load_dataset("Babelscape/REDFM", "all_languages", streaming=True, trust_remote_code=True)

for split in huggingface_ds:
    all_data.extend(process_file(huggingface_ds[split]))

# Convert to DataFrame all at once at the end
clean_data = pd.DataFrame(all_data)
pickle.dump(clean_data, open("../data/Evaluation Data/REDFM/processed_dataframe.pkl", "wb"))

In [ ]:
data = pickle.load(open("../data/Evaluation Data/REDFM/processed_dataframe.pkl", "rb"))
unique_ids = data['Correct QID'].unique()

data = pickle.load(open("../data/Evaluation Data/Mintaka/processed_dataframe.pkl", "rb"))
unique_ids = np.concatenate([unique_ids, pd.Series(np.concatenate(data['Question QIDs'].to_numpy())).unique()])
unique_ids = np.concatenate([unique_ids, pd.Series(np.concatenate(data['Answer QIDs'].to_numpy())).unique()])

data = pickle.load(open("../data/Evaluation Data/KGConv/processed_dataframe.pkl", "rb"))
unique_ids = np.concatenate([unique_ids, data['Question QID'].unique()])
unique_ids = np.concatenate([unique_ids, data['Answer QID'].unique()])

data = pickle.load(open("../data/Evaluation Data/Wikidata-Disamb/processed_dataframe.pkl", "rb"))
unique_ids = np.concatenate([unique_ids, data['Correct QID'].unique()])
unique_ids = np.concatenate([unique_ids, data['Wrong QID'].unique()])

unique_ids = pd.Series(unique_ids).unique()
unique_ids = pd.DataFrame({'QID': unique_ids})

In [ ]:
tqdm.pandas()

unique_ids['In Wikipedia'] = unique_ids['QID'].progress_apply(lambda x: (WikidataEntity.get_entity(x) is not None) and (WikidataID.get_id(x).in_wikipedia))

In [ ]:
test_data  = pd.DataFrame(columns=['Query', 'Correct', 'Source'])

data = pickle.load(open("../data/Evaluation Data/REDFM/processed_dataframe.pkl", "rb"))
prep = data[data['Language'] == 'en'][data['Correct in Wikipedia']]
prep['Correct QID'] = prep['Correct QID'].apply(lambda x: [x])
prep = prep.rename({'Sentence': 'Query', 'Correct QID': 'Correct'}, axis=1)
prep['Source'] = 'REDFM'

test_data = pd.concat([test_data, prep[['Query', 'Correct', 'Source']]])

def mintaka_filter(x):
    in_wiki = x['Question in Wikipedia'] + x['Answer in Wikipedia']
    if len(in_wiki) == 0:
        return False
    return all(in_wiki)

data = pickle.load(open("../data/Evaluation Data/Mintaka/processed_dataframe.pkl", "rb"))
prep = data[data.apply(lambda x: all(x['Question in Wikipedia'] + x['Answer in Wikipedia']), axis=1)]
prep['Correct'] = prep.apply(lambda x: x['Question QIDs'] + x['Answer QIDs'], axis=1)
prep = prep.rename({'Question': 'Query'}, axis=1)
prep['Source'] = 'Mintaka'

test_data = pd.concat([test_data, prep[['Query', 'Correct', 'Source']]])

data = pickle.load(open("../data/Evaluation Data/KGConv/processed_dataframe.pkl", "rb"))
prep = data[data.apply(lambda x: x['Question in Wikipedia'] and x['Answer in Wikipedia'], axis=1)]
prep['Correct'] = prep.apply(lambda x: [x['Question QID'], x['Answer QID']], axis=1)
prep = prep.rename({'Question': 'Query'}, axis=1)
prep['Source'] = 'KGConv'

test_data = pd.concat([test_data, prep[['Query', 'Correct', 'Source']]])
test_data

In [ ]:
def remove_spans(sentence, spans, replace_with='Entity'):
    # Sort spans in ascending order to remove from left to right
    spans = sorted(spans, key=lambda x: x[0])
    offset = 0  # To track the shift in index after replacing each span

    for start, end in spans:
        sentence = sentence[:start - offset] + replace_with + sentence[end - offset:]
        offset += (end - start) - len(replace_with)  # Update offset to account for the replaced span length

    return sentence

data['Sentence no entity'] = data.apply(lambda x: remove_spans(x['Sentence'], x['Entity Span']), axis=1)